# One sample test on simulated bulk RNA-seq

Nuha BinTayyash, 2020

This notebook shows how to run GPcounts with one sample test on simulated bulk RNA-seq datasets.

In [1]:
import numpy as np
import pandas as pd
from GPcounts.GPcounts_Module import Fit_GPcounts
import gpflow

In [2]:
files = ['low_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv'
         ,'low_counts_high_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv'                                      
         ,'high_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv'
         ,'high_counts_high_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv']

X = pd.read_csv('time_points.csv',index_col=[0])

for file in files:
    print(file)
    Y = pd.read_csv(file,index_col=[0])
    gp_counts = Fit_GPcounts(X,Y) 
    likelihoods = ['Negative_binomial','Gaussian','Poisson'] #
    for likelihood in likelihoods:
        log_likelihood = gp_counts.One_sample_test(likelihood)
        log_likelihood.to_csv("ll_"+likelihood+"_"+file)

  0%|          | 0/358 [00:00<?, ?it/s]

low_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv


  0%|          | 0/358 [00:00<?, ?it/s]

low_counts_high_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv


  0%|          | 0/360 [00:00<?, ?it/s]

high_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv


  0%|          | 0/360 [00:00<?, ?it/s]

high_counts_high_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv


100%|██████████| 360/360 [12:27<00:00,  2.08s/it]
